In [ ]:
import sys
import os
os.environ['USE_PYGEOS'] = '0'
import gc
import collections
from pathlib import Path

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd
from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import xarray as xr
from xeofs.xarray import EOF
import rioxarray

import dask
import distributed

import scipy
from scipy import signal
import cftime
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd
import cmocean

In [ ]:
# FIXIT: Jupyter Lab scheduler view for Dask not working
client = distributed.Client()
client

In [ ]:
# File path directories

# inDirName = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Get full path of the aislens_emulation directory. All file IO is relative to this path.
main_dir = Path.cwd().parent

DIR_external = 'data/external/'
DIR_processed = 'data/processed/'
DIR_interim = 'data/interim/'

# DATASET FILEPATHS
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# INTERIM GENERATED FILEPATHS
DIR_SORRMv21_Interim = 'data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/'

# DATA FILENAMES
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'

# Ice shelf basin/catchment definitions
FILE_iceShelvesShape = 'iceShelves.geojson'

In [ ]:
FILE_SORRMv21_RAW = 'SORRMv2.1.ISMF.nc'

In [ ]:
yr1 = 300
yr2 = 900
SORRMv21 = xr.open_dataset(main_dir.parent / 'aislens_emulation/' / DIR_external / 'SORRMv2.1.ISMF/regridded_output/' / FILE_SORRMv21, chunks={"Time":36})
SORRMv21_flux = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[yr1*12:yr2*12]
SORRMv21_draft = SORRMv21.timeMonthly_avg_ssh

In [ ]:
sorrmv21_raw = xr.open_dataset('/Users/smurugan9/research/aislens/aislens_emulation/data/external/SORRMv2.1.ISMF/tmp-folder-move-files/SORRMv2.1.ISMF.nc')

In [ ]:
SORRMv21_flux.max().compute()

In [ ]:
SORRMv21_flux.min().compute()

In [ ]:
SORRMv21_draft.min().compute()

In [ ]:
plt.figure()

plt.ylim(-2000,0)


# Draft Dependence of ice shelves
This section considers the relationship between freshwater flux and draft (`ssh` variable from the ocean model output is used as a proxy here)
Scatterplots for all the ice shelf regions are plotted using `src/plot_draft_dependence.py`


In [ ]:
#draft_params = xr.open_dataset(main_dir.parent / 'aislens_emulation/' / DIR_interim / 'SORRMv2.1.ISMF/iceShelves_dedraft/iceShelfRegions/iceshelves_draft_dependence_parameters_total.nc')
draft_params = xr.open_dataset(main_dir.parent / 'aislens_emulation/' / DIR_interim / 'SORRMv2.1.ISMF/iceShelves_dedraft/ICESHELVES_AIS_RGRS.nc')

In [ ]:
draft_params